# Лабораторна робота №10
## Тема: Стиснення даних. Жадібна стратегія на прикладі кодування Гафмена
#### Виконав: Студент групи КН-24-1 Соломка Борис Олегович

## Вступ

**Мета роботи:** навчитись реалізовувати алгоритм побудови дерева оптимальних кодів Гафмена на основі черги з пріоритетом (за допомогою купи) засобами Python.

**Завдання:**
1. Використати можливості бібліотеки heapq
2. Реалізувати чергу з пріоритетами засобами мови Python
3. Реалізувати алгоритм оптимального кодування Гафмена
4. Реалізувати алгоритм декодування Гафмена

## Теоретичні відомості

**Кодування Гафмена** – алгоритм стиснення даних без втрат, що присвоює вхідним символам коди змінної довжини залежно від частоти їх появи. Ці коди є префіксними, тобто код одного символу не є префіксом коду іншого символу, що гарантує однозначність декодування.

**Основні етапи кодування Гафмена:**
1. Побудова дерева Гафмена на основі вхідних символів
2. Проходження по дереву та присвоювання кодів символам

**Алгоритм побудови дерева:**
1. Створити вершину для кожного символу і побудувати мінімальну купу
2. Витягнути два вузли з найменшою частотою
3. Створити новий вузол із сумарною частотою і додати його до купи
4. Повторювати кроки 2-3 до отримання одного вузла в купі

## Хід роботи

### 1. Базова реалізація алгоритму кодування Гафмена

In [1]:
# A Вузол дерева Гафмена
import heapq

class node:
    def __init__(self, freq, symbol, left=None, right=None):
        # частота символу
        self.freq = freq
        # назва символу
        self.symbol = symbol
        # вузол ліворуч від поточного вузла
        self.left = left
        # вузол праворуч від поточного вузла
        self.right = right
        # напрямок дерева (0/1)
        self.huff = ''
        
    def __lt__(self, nxt):
        return self.freq < nxt.freq

In [2]:
# Функція для виведення кодів Гафмена
def printNodes(node, val=''):
    # Код Гафмена для поточного вузла
    newVal = val + str(node.huff)
    
    # якщо вершина не є реберною вершиною
    # то пройти всередині неї
    if(node.left):
        printNodes(node.left, newVal)
    if(node.right):
        printNodes(node.right, newVal)
        
    # якщо node є реберною вершиною тоді
    # вивести його хаффманівський код
    if(not node.left and not node.right):
        print(f"{node.symbol} -> {newVal}")

In [3]:
# символи для дерева Гафмена
chars = ['a', 'b', 'c', 'd', 'e', 'f']

# частота символів
freq = [5, 9, 12, 13, 16, 45]

# список, що містить невикористані вершини
nodes = []

# перетворення символів та частот у вузли дерева Гафмена
for x in range(len(chars)):
    heapq.heappush(nodes, node(freq[x], chars[x]))

while len(nodes) > 1:
    # відсортувати всі вершини за зростанням на основі їх частоти
    left = heapq.heappop(nodes)
    right = heapq.heappop(nodes)
    
    # присвоїти значення напрямку цим вузлам
    left.huff = 0
    right.huff = 1
    
    # об'єднати 2 найменші вершини, щоб створити
    # новий вузол як їхній батько
    newNode = node(left.freq+right.freq, left.symbol+right.symbol, left, right)
    heapq.heappush(nodes, newNode)

# Друк кодів Гафмена
printNodes(nodes[0])

f -> 0
c -> 100
d -> 101
a -> 1100
b -> 1101
e -> 111


### 2. Розширення алгоритму для обчислення частот у повідомленні

In [4]:
def calculate_frequencies(message):
    """
    Функція для обчислення частоти символів у повідомленні
    
    Параметри:
    message (str): Вхідне повідомлення
    
    Повертає:
    tuple: (список символів, список їх частот)
    """
    # Словник для збереження частот
    freq_dict = {}
    
    # Обчислення частот
    for char in message:
        if char in freq_dict:
            freq_dict[char] += 1
        else:
            freq_dict[char] = 1
    
    # Створення списків символів та частот
    chars = list(freq_dict.keys())
    freq = [freq_dict[char] for char in chars]
    
    return chars, freq

In [5]:
# Приклад використання функції
message = "Hello world!"
chars, freq = calculate_frequencies(message)

print(f"Символи: {chars}")
print(f"Частоти: {freq}")

Символи: ['H', 'e', 'l', 'o', ' ', 'w', 'r', 'd', '!']
Частоти: [1, 1, 3, 2, 1, 1, 1, 1, 1]


### 3. Реалізація алгоритму декодування Гафмена

In [6]:
def huffman_encoding(message):
    """
    Функція для кодування повідомлення за алгоритмом Гафмена
    
    Параметри:
    message (str): Повідомлення для кодування
    
    Повертає:
    tuple: (закодоване повідомлення, словник кодів)
    """
    # Отримання символів та їх частот
    chars, freq = calculate_frequencies(message)
    
    # Список вузлів
    nodes = []
    
    # Створення вузлів для кожного символу
    for i in range(len(chars)):
        heapq.heappush(nodes, node(freq[i], chars[i]))
    
    # Побудова дерева Гафмена
    while len(nodes) > 1:
        left = heapq.heappop(nodes)
        right = heapq.heappop(nodes)
        
        left.huff = 0
        right.huff = 1
        
        new_node = node(left.freq + right.freq, left.symbol + right.symbol, left, right)
        heapq.heappush(nodes, new_node)
    
    # Словник для збереження кодів
    huffman_codes = {}
    
    # Функція для отримання кодів
    def get_codes(node, val=''):
        new_val = val + str(node.huff)
        
        if node.left:
            get_codes(node.left, new_val)
        if node.right:
            get_codes(node.right, new_val)
            
        if not node.left and not node.right:
            huffman_codes[node.symbol] = new_val
    
    # Отримання кодів
    get_codes(nodes[0])
    
    # Кодування повідомлення
    encoded_message = ''
    for char in message:
        encoded_message += huffman_codes[char]
    
    return encoded_message, huffman_codes

In [7]:
def huffman_decoding(encoded_message, huffman_codes):
    """
    Функція для декодування повідомлення закодованого за алгоритмом Гафмена
    
    Параметри:
    encoded_message (str): Закодоване повідомлення
    huffman_codes (dict): Словник кодів Гафмена
    
    Повертає:
    str: Декодоване повідомлення
    """
    # Створення зворотного словника кодів
    reverse_codes = {code: char for char, code in huffman_codes.items()}
    
    # Декодування повідомлення
    decoded_message = ''
    current_code = ''
    
    for bit in encoded_message:
        current_code += bit
        if current_code in reverse_codes:
            decoded_message += reverse_codes[current_code]
            current_code = ''
    
    return decoded_message

In [8]:
# Приклад використання функцій кодування та декодування
message = "Hello world!"
encoded_message, huffman_codes = huffman_encoding(message)
decoded_message = huffman_decoding(encoded_message, huffman_codes)

print(f"Оригінальне повідомлення: {message}")
print(f"Закодоване повідомлення: {encoded_message}")
print(f"Словник кодів: {huffman_codes}")
print(f"Декодоване повідомлення: {decoded_message}")

Оригінальне повідомлення: Hello world!
Закодоване повідомлення: 001010111111001010001110
Словник кодів: {'H': '00', 'e': '01', 'l': '10', 'o': '11', ' ': '000', 'w': '001', 'r': '0010', 'd': '0011', '!': '111'}
Декодоване повідомлення: Hello world!


### 4. Візуалізація дерева Гафмена для прикладу з лабораторної роботи

In [9]:
# Функція для візуалізації дерева Гафмена (текстове представлення)
def visualize_huffman_tree(root, prefix='', is_left=True):
    if root is not None:
        # Формування префіксу для поточного вузла
        if prefix != '':
            connector = '├── ' if is_left else '└── '
        else:
            connector = 'Корінь '
        
        # Вивід інформації про вузол
        if root.left is None and root.right is None:
            print(f"{prefix}{connector}{root.huff}: {root.symbol} ({root.freq})")
        else:
            print(f"{prefix}{connector}({root.freq}): {root.symbol}")
        
        # Рекурсивний виклик для дочірніх вузлів
        if root.left is not None or root.right is not None:
            new_prefix = prefix + ('│   ' if is_left else '    ')
            if root.left:
                visualize_huffman_tree(root.left, new_prefix, True)
            if root.right:
                visualize_huffman_tree(root.right, new_prefix, False)

# Повторне створення дерева з прикладу
chars = ['a', 'b', 'c', 'd', 'e', 'f']
freq = [5, 9, 12, 13, 16, 45]
nodes = []

for x in range(len(chars)):
    heapq.heappush(nodes, node(freq[x], chars[x]))

while len(nodes) > 1:
    left = heapq.heappop(nodes)
    right = heapq.heappop(nodes)
    
    left.huff = 0
    right.huff = 1
    
    newNode = node(left.freq+right.freq, left.symbol+right.symbol, left, right)
    heapq.heappush(nodes, newNode)

# Візуалізація дерева
print("Дерево Гафмена (текстове представлення):")
print()
visualize_huffman_tree(nodes[0], '')
print()
print("Коди символів:")
printNodes(nodes[0])

Дерево Гафмена (текстове представлення):

Корінь (100): abcdef
├── 0: f (45)
└── 1: abcde (55)
    ├── 0: abcd (39)
    │   ├── 0: ab (14)
    │   │   ├── 0: a (5)
    │   │   └── 1: b (9)
    │   └── 1: cd (25)
    │       ├── 0: c (12)
    │       └── 1: d (13)
    └── 1: e (16)

Коди символів:
f -> 0
a -> 1000
b -> 1001
c -> 1010
d -> 1011
e -> 11


## Відповіді на контрольні питання

### 1. Що таке жадібні алгоритми?

Жадібні алгоритми - це алгоритми, які на кожному кроці обирають локально оптимальне рішення, сподіваючись, що це приведе до глобально оптимального рішення. Вони не переглядають попередні рішення, а приймають рішення на основі поточної інформації. Жадібні алгоритми ефективні для певних класів задач, але не гарантують оптимальності для всіх задач.

### 2. Що таке префіксний код? Який код використовується у коді Гафмена?

Префіксний код - це код, у якому жоден код не є префіксом (початком) іншого коду. Це забезпечує однозначність декодування. У кодуванні Гафмена використовується саме префіксний код, що дозволяє декодувати повідомлення без додаткових роздільників між кодами символів.

### 3. Як пов'язана структура даних «купа» зі структурою даних «черга з пріоритетами»?

Купа (heap) є ефективною реалізацією черги з пріоритетами. Черга з пріоритетами - це абстрактна структура даних, яка дозволяє додавати елементи та видаляти елемент з найвищим (або найнижчим) пріоритетом. Купа реалізує ці операції з логарифмічною складністю, що робить її оптимальною для реалізації черги з пріоритетами.

### 4. Що таке стиснення даних і для чого воно використовується? Які його основні переваги?

Стиснення даних - це процес кодування інформації з використанням меншої кількості бітів, ніж в оригінальному представленні. Використовується для:
- Зменшення обсягу даних при зберіганні
- Прискорення передачі даних по мережі
- Економії ресурсів пам'яті

Основні переваги:
- Економія простору для зберігання
- Збільшення швидкості передачі даних
- Зменшення витрат на зберігання та передачу даних

### 5. Які кроки необхідно виконати для стиснення даних за допомогою алгоритму кодування Гафмена?

1. Обчислити частоту кожного символу в повідомленні
2. Побудувати мінімальну купу (чергу з пріоритетами) на основі частот
3. Побудувати дерево Гафмена, об'єднуючи два вузли з найменшими частотами
4. Присвоїти коди кожному символу, проходячи дерево (0 для лівої гілки, 1 для правої)
5. Закодувати повідомлення, замінюючи кожен символ відповідним кодом

### 6. Які основні обмеження та недоліки алгоритму кодування Гафмена? Чи можливо покращити його продуктивність?

Обмеження та недоліки:
- Необхідність знати частоти символів заздалегідь
- Потреба зберігати дерево або таблицю кодів для декодування
- Не завжди оптимальний для дуже коротких повідомлень
- Обмежена ефективність для даних з рівномірним розподілом частот

Покращення продуктивності:
- Використання адаптивного кодування Гафмена
- Поєднання з іншими методами стиснення (наприклад, RLE)
- Оптимізація структур даних для зберігання дерева

### 7. Які існують альтернативні методи стиснення даних, що можуть конкурувати з алгоритмом Гафмена?

Альтернативні методи:
- Арифметичне кодування (краще стиснення, але повільніше)
- Алгоритм Лемпеля-Зіва (LZ77, LZ78, LZSS)
- Алгоритм Берроуза-Уілера (BWT)
- Алгоритм PPM (Prediction by Partial Matching)
- Алгоритм RLE (Run-Length Encoding) для простих типів даних
- Методи на основі словників (LZW, DEFLATE)

### 8. Які практичні застосування можуть мати алгоритми стиснення даних, зокрема алгоритм Гафмена, у сучасних інформаційних системах?

Практичні застосування:
- Стиснення файлів (архіватори ZIP, GZIP, RAR)
- Стиснення мультимедійних форматів (JPEG, PNG, MP3)
- Передача даних у телекомунікаційних системах
- Зменшення розміру баз даних
- Оптимізація використання пам'яті в вбудованих системах


## Висновки

У ході виконання лабораторної роботи було досліджено та реалізовано алгоритм кодування та декодування Гафмена, який є ефективним методом стиснення даних без втрат.

Основні результати роботи:

1. Вивчено принципи роботи алгоритму Гафмена та його теоретичні основи.
2. Розроблено та протестовано реалізацію побудови дерева Гафмена з використанням черги з пріоритетом (на основі бібліотеки heapq).
3. Реалізовано функцію для обчислення частот символів у вхідному повідомленні.
4. Розроблено алгоритми кодування та декодування повідомлень за допомогою кодів Гафмена.
5. Створено текстову візуалізацію дерева Гафмена для кращого розуміння його структури.

Часова складність алгоритму кодування Гафмена становить O(n log n), де n - кількість унікальних символів, що робить його ефективним для практичного застосування. Алгоритм Гафмена є одним із класичних прикладів застосування жадібної стратегії, що в даному випадку призводить до оптимального результату у вигляді префіксного коду з мінімальною середньою довжиною.

Практичне значення цієї роботи полягає в розумінні основ стиснення даних та принципів роботи сучасних систем архівації та обробки інформації.